In [13]:
import pandas as pd
import os

# Directory paths for cleaned and joined folders
CLEANED_FOLDER = 'cleaned'
JOINED_FOLDER = 'joined'

# Ensure the 'joined' folder exists
os.makedirs(JOINED_FOLDER, exist_ok=True)

def join_tables():
    """Join all cleaned CSV files into one DataFrame."""
    # Load cleaned CSV files into DataFrames
    certificates_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'certificates_cleaned.csv'))
    course_assignments_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'course_assignments_cleaned.csv'))
    performance_ratings_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'performance_ratings_cleaned.csv'))
    performance_summaries_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'performance_summaries_cleaned.csv'))
    users_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'users.csv'))
    courses_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'courses.csv'))
    learning_paths_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'learning_paths.csv'))
    course_learning_paths_df = pd.read_csv(os.path.join(CLEANED_FOLDER, 'course_learning_paths.csv'))

    # Print columns for debugging
    print("Certificates Columns:", certificates_df.columns.tolist())
    print("Course Assignments Columns:", course_assignments_df.columns.tolist())
    print("Performance Ratings Columns:", performance_ratings_df.columns.tolist())
    print("Performance Summaries Columns:", performance_summaries_df.columns.tolist())
    print("Users Columns:", users_df.columns.tolist())
    print("Courses Columns:", courses_df.columns.tolist())
    print("Learning Paths Columns:", learning_paths_df.columns.tolist())
    print("Course Learning Paths Columns:", course_learning_paths_df.columns.tolist())

    # Proceed with merging
    merged_df = (
        certificates_df
        .merge(course_assignments_df, on='user_id', how='left', suffixes=('', '_ca'))
    )

    print("After merging certificates and course assignments:")
    print(merged_df.columns.tolist())
    print(merged_df.shape)

    merged_df = merged_df.merge(performance_ratings_df, on=['user_id', 'assignment_id'], how='left', suffixes=('', '_pr'))

    print("After merging with performance ratings:")
    print(merged_df.columns.tolist())
    print(merged_df.shape)

    # Remove duplicate 'id' column from performance ratings
    merged_df = merged_df.drop(columns=['id_pr'], errors='ignore')

    # Attempt to merge with course learning paths
    merged_df = merged_df.merge(course_learning_paths_df, on='course_id', how='left', suffixes=('', '_clp'))

    print("After merging with course learning paths:")
    print(merged_df.columns.tolist())
    print(merged_df.shape)

    # Remove duplicate 'id' column from course learning paths
    merged_df = merged_df.drop(columns=['id_clp'], errors='ignore')

    # Join performance summaries with additional context for learning_path_id
    merged_df = merged_df.merge(performance_summaries_df, on=['user_id', 'learning_path_id'], how='left', suffixes=('', '_ps'))

    # Continue merging with users, courses, and learning paths
    merged_df = (
        merged_df
        .merge(users_df, left_on='user_id', right_on='id', how='left', suffixes=('', '_user'))
        .merge(courses_df, left_on='course_id', right_on='id', how='left', suffixes=('', '_course'))
        .merge(learning_paths_df, left_on='learning_path_id', right_on='id', how='left', suffixes=('', '_lp'))
    )

    # Drop duplicate columns and keep only the first occurrence
    merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

    # Save the joined DataFrame to a CSV file
    merged_df.to_csv(os.path.join(JOINED_FOLDER, 'joined_data.csv'), index=False)
    print("Joined data saved to 'joined_data.csv'.")

# Join tables
join_tables()

print("Data merging completed.")


Certificates Columns: ['id', 'is_certified', 'user_id', 'assignment_id']
Course Assignments Columns: ['id', 'user_id', 'course_id', 'progress', 'assignment_date', 'quiz_score', 'engagement_score', 'assignment_grade']
Performance Ratings Columns: ['id', 'rating', 'user_id', 'assignment_id']
Performance Summaries Columns: ['id', 'user_id', 'average_rating', 'learning_path_id']
Users Columns: ['id', 'name', 'designation', 'role']
Courses Columns: ['id', 'title', 'duration', 'difficulty Level']
Learning Paths Columns: ['id', 'title', 'description']
Course Learning Paths Columns: ['id', 'course_id', 'learning_path_id']
After merging certificates and course assignments:
['id', 'is_certified', 'user_id', 'assignment_id', 'id_ca', 'course_id', 'progress', 'assignment_date', 'quiz_score', 'engagement_score', 'assignment_grade']
(156583, 11)
After merging with performance ratings:
['id', 'is_certified', 'user_id', 'assignment_id', 'id_ca', 'course_id', 'progress', 'assignment_date', 'quiz_score'